In [2]:
%pip install torch


  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   - -------------------------------------- 9.2/212.5 MB 47.4 MB/s eta 0:00:05
   --- ------------------------------------ 19.4/212.5 MB 48.9 MB/s eta 0:00:04
   ---- ----------------------------------- 25.2/212.5 MB 40.9 MB/s eta 0:00:05
   ----- ---------------------------------- 27.8/212.5 MB 35.9 MB/s eta 0:00:06
   ----- ---------------------------------- 31.7/212.5 MB 31.0 MB/s eta 0:00:06
   ------ --------------------------------- 35.4/212.5 MB 28.5 MB/s eta 0:00:07
   ------- -------------------------------- 41.4/212.5 MB 28.9 MB/s eta 0:00:06
   --------- ------------------------------ 51.6/212.5 MB 31.3 MB/s eta 0:00:06
   ----------- ---------------------------- 60.8/212.5 MB 32.8 MB/s eta 0:00:05
   ------------ ---

In [7]:
import torch, pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import numpy as np
import pandas as pd


data = pd.read_csv("data/gym_churn.csv")


class DNN(nn.Module):
    def __init__(self, Cin):
        super().__init__()
        self.relu = nn.ReLU()

        self.dropout = nn.Dropout(0.4)
        self.linear1 = nn.Linear(Cin, 32)
        self.linear2 = nn.Linear(32, 128)
        self.linear3 = nn.Linear(128, 16)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc(out)

        return out


# X = torch.tensor(data.drop('Churn', axis=1).to_numpy(), dtype=torch.float32) #(3200, 12)
X = data.drop('Churn', axis=1)
y = torch.tensor(data['Churn'].to_numpy(), dtype=torch.float32).unsqueeze(dim=1) #(3200, 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

# scaler = StandardScaler()
scaler = RobustScaler()
X_train_scaled = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_val_scaled =  torch.tensor(scaler.transform(X_val), dtype=torch.float32)

model = DNN(Cin = X_train_scaled.size(1))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr = 0.001)#, weight_decay=1e-4)

model.train()
best_f1score=  0.
for epoch in range(1000):

  optimizer.zero_grad()
  y_pred = model(X_train_scaled).type(torch.float32)
  loss = criterion(y_pred, y_train)
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 10 == 0:
    print(f'Epoch: {epoch + 1} Loss: {loss.item()}')

  if (epoch + 1) % 10 == 0:

    model.eval()
    with torch.no_grad():

      y_val_pred = model(X_val_scaled)
      loss = criterion(y_val_pred, y_val)

      y_val_prob = F.sigmoid(y_val_pred)
      y_val_pred_label = (y_val_prob >= 0.5).float()

      y_true = y_val.numpy()
      y_pred_labels = y_val_pred_label.numpy()

      acc = accuracy_score(y_true, y_pred_labels)
      precision = precision_score(y_true, y_pred_labels)
      recall = recall_score(y_true, y_pred_labels)
      f1 = f1_score(y_true, y_pred_labels)

      print(f'EVAL loss: {loss:.4f} Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

      if best_f1score < f1:
        torch.save(model.state_dict(), './best_model.pth')
        with open('DNN_scaler.pkl', 'wb') as f:
          pickle.dump(scaler, f)
        print(f'====Best model saved at {epoch+1} with F1 Score: {f1:.4f}')
        best_f1score = f1




Epoch: 10 Loss: 0.6599882245063782
EVAL loss: 0.6495 Accuracy: 0.7550, Precision: 1.0000, Recall: 0.0755, F1 Score: 0.1404
====Best model saved at 10 with F1 Score: 0.1404
Epoch: 20 Loss: 0.5691842436790466
EVAL loss: 0.5530 Accuracy: 0.7350, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch: 30 Loss: 0.4664098620414734
EVAL loss: 0.4507 Accuracy: 0.7350, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch: 40 Loss: 0.39309462904930115
EVAL loss: 0.3814 Accuracy: 0.7700, Precision: 0.9667, Recall: 0.1368, F1 Score: 0.2397
====Best model saved at 40 with F1 Score: 0.2397
Epoch: 50 Loss: 0.3289259374141693
EVAL loss: 0.3211 Accuracy: 0.8775, Precision: 0.9385, Recall: 0.5755, F1 Score: 0.7135
====Best model saved at 50 with F1 Score: 0.7135
Epoch: 60 Loss: 0.2864466905593872
EVAL loss: 0.2837 Accuracy: 0.9087, Precision: 0.8840, Recall: 0.7547, F1 Score: 0.8142
====Best model saved at 60 with F1 Score: 0.8142


C:\Users\play data\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\play data\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 70 Loss: 0.25237706303596497
EVAL loss: 0.2535 Accuracy: 0.9087, Precision: 0.8424, Recall: 0.8066, F1 Score: 0.8241
====Best model saved at 70 with F1 Score: 0.8241
Epoch: 80 Loss: 0.22464916110038757
EVAL loss: 0.2267 Accuracy: 0.9150, Precision: 0.8429, Recall: 0.8349, F1 Score: 0.8389
====Best model saved at 80 with F1 Score: 0.8389
Epoch: 90 Loss: 0.20406000316143036
EVAL loss: 0.2062 Accuracy: 0.9200, Precision: 0.8558, Recall: 0.8396, F1 Score: 0.8476
====Best model saved at 90 with F1 Score: 0.8476
Epoch: 100 Loss: 0.18962350487709045
EVAL loss: 0.1935 Accuracy: 0.9275, Precision: 0.8667, Recall: 0.8585, F1 Score: 0.8626
====Best model saved at 100 with F1 Score: 0.8626
Epoch: 110 Loss: 0.18031135201454163
EVAL loss: 0.1868 Accuracy: 0.9325, Precision: 0.8798, Recall: 0.8632, F1 Score: 0.8714
====Best model saved at 110 with F1 Score: 0.8714
Epoch: 120 Loss: 0.17353500425815582
EVAL loss: 0.1828 Accuracy: 0.9275, Precision: 0.8738, Recall: 0.8491, F1 Score: 0.8612
Epoch: